## Tests for Ishigami function
Exact permutation. Analytical Shapley values obtained from Plischke, Rabitti, and Borgonovo (2020), p. 8. Inputs are independent.

In [1]:
import numpy as np
import pandas as pd
import chaospy as cp
import math
from numpy.testing import assert_array_almost_equal as aaae

from econsa_shapley import get_shapley
from econsa_shapley import _r_condmvn
from econsa_sampling import cond_mvn

In [2]:
# What does x_cond from the test cases spit out?
def x_cond_linda(n, subset_j, subsetj_conditional, xjc):
    if subsetj_conditional is None:
        cov_int = np.array(cov)  # Get an array from cov.
        cov_int = cov_int.take(subset_j, axis=1)  # Take only columns/rows (depending on axis) that correspond to the indices
        #provided by subset_j.
        cov_int = cov_int[subset_j]
        distribution = cp.MvNormal(mean[subset_j], cov_int)  # Take mean only for indices in sj.
        return distribution.sample(n)
    else:
        return _r_condmvn(
            n,
            mean=mean,
            cov=cov,
            dependent_ind=subset_j,
            given_ind=subsetj_conditional,
            x_given=xjc,
            )
    

In [3]:
def test_get_shapley_ishigami():
    def ishigami_function(x):
        return math.sin(x[:, 1]) * (1 + 0.1 * (x[:, 3] ** 4)) + 7 * (math.sin(x[:, 2]) ** 2)

    def x_all(n):
        distribution = cp.Iid(cp.Uniform(lower, upper), n_inputs)
        return distribution.sample(n)


    np.random.seed(123)
    n_inputs = 3
    mean = np.zeros(3)
    # Lower and upper bound of the uniform distribution.
    lower = -math.pi
    upper = math.pi
    variance = (1/12) * ((upper - lower) ** 2)
    cov = np.array([[1.0, 0, 0], [0, 1.0, 1.8], [0, 1.8, 4.0]])
    method = "exact"
    n_perms = None
    n_output = 10 ** 4
    n_outer = 10 ** 3
    n_inner = 10 ** 2

    col = ["X" + str(i) for i in np.arange(n_inputs) + 1]
    names = ["Shapley effects", "std. errors", "CI_min", "CI_max"]

    expected = pd.DataFrame(
        data=[
            [0.4358, 0.4424, 0.1218],
            [0., 0., 0.],
            [0., 0., 0.],
            [0., 0., 0.],
        ],
        index=names,
        columns=col,
    ).T

    calculated = get_shapley(
        method,
        gaussian_model,
        x_all,
        x_cond,
        n_perms,
        n_inputs,
        n_output,
        n_outer,
        n_inner,
    )

    aaae(calculated['Shapley effects'], expected['Shapley effects'])

### Get Shapley values for the model with the Ishigami function. 

In [4]:
# Number of permutations will be k! if the exact method is applied.
n_inputs = 3
n_output = 10 ** 4
n_outer = 10 ** 3
n_inner = 10 ** 2
n_perms = math.factorial(n_inputs)

In [58]:
model_inputs = np.array([[1, 2, 3,],
                        [4, 5, 7],
                        [3, 1, 6],
                        [4, 3, 2]])

In [25]:
for i in model_inputs:
    out = i[0]
    print(out)

1
4
3
4


In [ ]:
def ishigami_with_np_sin(x):
    

In [61]:
def ishigami_function_draft(x):
    n_samples = len(model_inputs)  # n_samples needs to be long enough! See shapley.py for dim.s!
    out = np.zeros((n_samples, 1))
    for i in range(n_samples):
        x_1 = x[i, 0]
        x_2 = x[i, 1]
        x_3 = x[i, 2]
        out[i, :] = math.sin(x_1) * (1 + 0.1 * (x_3 ** 4)) + 7 * (math.sin(x_2) ** 2)
    np.array([your_function(i) for i in x])
    return out

In [62]:
ishigami_function_draft(model_inputs)

array([[  13.44513863],
       [-176.02833127],
       [  23.38678698],
       [  -1.82828249]])

In [11]:
def x_all(n):
    distribution = cp.Iid(cp.Uniform(lower, upper), n_inputs)
    return distribution.sample(n)

In [112]:
def ishigami_function(x):
    def execute_ishigami(a, b, c):
        out = np.sin(a) * (1 + 0.1 * (c ** 4)) + 7 * (np.sin(b) ** 2)
        return out
    out_2 = execute_ishigami(*x.T)
    return out_2

In [113]:
ishigami_function(model_inputs)

array([  13.44513863, -176.02833127,   23.38678698,   -1.82828249])

In [110]:
def execute_ishigami_draft(a, b, c):
        out = np.sin(a) * (1 + 0.1 * (c ** 4)) + 7 * (np.sin(b) ** 2)
        return out

In [111]:
execute_ishigami_draft(*model_inputs.T)

array([  13.44513863, -176.02833127,   23.38678698,   -1.82828249])

In [104]:
def x_cond(n, subset_j, subsetj_conditional, xjc):
        distribution = cp.Iid(cp.Uniform(lower, upper), len(subset_j))
        return distribution.sample(n)

In [126]:
exact_shapley = 3
n_output = 10 ** 6
n_outer = 10 ** 5
n_inner = 10 ** 2
lower = -np.pi
upper = np.pi
method = 'exact'
np.random.seed(1234)
n_perms = None

In [127]:
%%time
exact_shapley = get_shapley(method, ishigami_function, x_all, x_cond, n_perms, n_inputs, n_output, n_outer, n_inner)

Wall time: 46min 58s


In [128]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
X1,0.435551,0.049788,0.337967,0.533135
X2,0.444085,0.001330,0.441478,0.446692
X3,0.120364,0.050314,0.021748,0.218980


In [125]:
exact_shapley  # 4 Min, N_O = 10^5

,Shapley effects,std. errors,CI_min,CI_max
X1,0.432367,0.048797,0.336724,0.528010
X2,0.443831,0.000970,0.441929,0.445732
X3,0.123802,0.048919,0.027920,0.219684


In [117]:
exact_shapley

,Shapley effects,std. errors,CI_min,CI_max
X1,0.433179,0.048947,0.337243,0.529116
X2,0.446127,0.003176,0.439902,0.452351
X3,0.120694,0.050359,0.021991,0.219397


In [120]:
effetti_shapley_analitici = [0.4358, 0.4424, 0.1218]